In [34]:
from scipy.stats import norm
from scipy.optimize import brentq
import math
import pandas as pd 
import numpy as np
import json

import requests
from datetime import datetime
from dotenv import load_dotenv
import os
load_dotenv()
REBAR_API_KEY = os.getenv('REBAR_API_KEY')
COINGECKO_API_KEY = os.getenv('COINGECKO_API_KEY')

In [ ]:
#coingecko for prices
COINGECKO_URL = "https://api.coingecko.com/api/v3"
COINGECKO_HEADERS = {
    "x-cg-demo-api-key": COINGECKO_API_KEY
}
def get_coins():
    url = f"{COINGECKO_URL}/coins/list"
    res = requests.get(url,headers=COINGECKO_HEADERS)
    return res.json()

def get_coin_historical_data(id:str, date:str="19-05-2025"):
    url = f"{COINGECKO_URL}/coins/{id}/history?date={date}"
    res = requests.get(url,headers=COINGECKO_HEADERS)
    return res.json()

def get_coin_prices(id:str, vs_currency:str="btc", days:str="90", interval="daily"):
    url = f"{COINGECKO_URL}/coins/{id}/market_chart?vs_currency={vs_currency}&days={days}&interval={interval}"
    res = requests.get(url,headers=COINGECKO_HEADERS)
    return res.json()

def calculate_volatility(price_data):
    """
    Calculate annualized historical volatility from price data.
    
    price_data: List of [timestamp, price] pairs (timestamps in ms)
    """
    prices = [p[1] for p in sorted(price_data, key=lambda x: x[0])]
    log_returns = np.diff(np.log(prices))
    volatility = np.std(log_returns) * math.sqrt(365)  # Annualized
    return volatility

def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    """
    Black-Scholes option pricing.
    
    S: Current price of underlying
    K: Strike price
    T: Time to maturity in years
    r: Risk-free interest rate (e.g., 0.05 for 5%)
    sigma: Volatility (standard deviation of returns, annualized)
    option_type: 'call' or 'put'
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    
    return price

def calculate_options_price(ticker:str="pepe", 
                            strike_price_multiple:float=1.2,
                            option_type:str="call",
                            expiry_in_years:float=0.5,     
                            risk_free_rate:float=0.05,
                            volatility_lookback_days:str="90",
                            vs_currency="usd"
                            ):
    prices = get_coin_prices(ticker, vs_currency=vs_currency, days=volatility_lookback_days)["prices"]
    
    current_price = prices[0][1]
    strike_price = current_price*strike_price_multiple
    sigma = calculate_volatility(prices)
    option_price = black_scholes_price(current_price,
                                                 strike_price, 
                                                 expiry_in_years, 
                                                 risk_free_rate, 
                                                 sigma,
                                                 option_type
                                                 )
    data = {
        "option_price":option_price,
        "prices":prices,
    }

    return data


In [58]:
pepe_prices = get_coin_prices("pepe", vs_currency="usd")
with open('pepe_prices_usd.json','w') as f:
    json.dump(pepe_prices, f,indent=2)

In [ ]:
S = pepe_prices["prices"][0][1]
K = S*1.2
T = 0.5
r = 0.05
sigma = calculate_volatility(pepe_prices["prices"])
option_price = black_scholes_price(S,K,T,r,sigma)
print(option_price)

1.6752522003849541e-07


In [62]:
data = calculate_options_price("pepe",1.5,"call",0.5,0.05,"90","usd")
print(data)

2.3052596848879498e-06
[[1740009600000, 9.35041819583723e-06], [1740096000000, 9.646776818017512e-06], [1740182400000, 9.109422651158587e-06], [1740268800000, 9.637858478068004e-06], [1740355200000, 9.368960514063803e-06], [1740441600000, 7.869360985077898e-06], [1740528000000, 8.241409099300332e-06], [1740614400000, 8.290925717677615e-06], [1740700800000, 8.215808716001352e-06], [1740787200000, 7.912161254834238e-06], [1740873600000, 7.587406866726591e-06], [1740960000000, 8.809667019135499e-06], [1741046400000, 7.220139107719918e-06], [1741132800000, 6.993281049870955e-06], [1741219200000, 7.080489729513542e-06], [1741305600000, 6.84477636683213e-06], [1741392000000, 6.977135210639283e-06], [1741478400000, 6.665449809895258e-06], [1741564800000, 5.857004408429998e-06], [1741651200000, 5.7578438493628235e-06], [1741737600000, 6.33497578699499e-06], [1741824000000, 6.939394327617562e-06], [1741910400000, 6.6986609398856236e-06], [1741996800000, 7.0634120216730535e-06], [1742083200000, 

In [31]:
#options
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    """
    Prices a European option using the Black-Scholes formula.

    Args:
        S (float): Current price of the underlying asset (spot price).
        K (float): Strike price of the option.
        T (float): Time to maturity in years.
        r (float): Risk-free interest rate (annualized).
        sigma (float): Volatility of the underlying asset (annualized).
        option_type (str): Either 'call' or 'put'.

    Returns:
        float: The theoretical option price.
    """
    if T <= 0 or sigma <= 0:
        return max(0.0, S - K) if option_type == 'call' else max(0.0, K - S)

    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type == 'call':
        return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    else:
        return K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def estimate_volatility(data):
    # Sort data by timestamp ascending
    data_sorted = sorted(data, key=lambda x: x['timestamp'])
    
    # Extract values as floats
    values = np.array([float(item['value']) for item in data_sorted])
    print(values)
    
    # Extract timestamps as float days (milliseconds to days)
    timestamps = np.array([item['timestamp'] for item in data_sorted])
    timestamp_minutes = (timestamps - timestamps[0]) / (1000 * 60)  # convert ms to days from first timestamp
    print(timestamp_minutes)
    # Calculate log returns between consecutive values
    log_returns = np.diff(np.log(values))
    print(log_returns)
    # Calculate time differences in days between consecutive timestamps
    dt = np.diff(timestamp_minutes)
    print(dt)
    # Compute annualized volatility
    # Weighted std of returns by time intervals, scaled by sqrt of days in a year (365)
    # Formula: std(log_returns / sqrt(dt)) * sqrt(365)
    # Here we annualize volatility by scaling returns by sqrt(dt) and then rescale to annual
    scaled_returns = log_returns / np.sqrt(dt)
    print(scaled_returns)
    volatility = np.std(scaled_returns) * math.sqrt(365)
    
    return volatility

def time_to_expiry(expiry_timestamp_ms):
    """
    Calculates the time to expiry in years.

    Args:
        expiry_timestamp_ms (int): Expiry timestamp in milliseconds.

    Returns:
        float: Time to expiry in years.
    """
    now = datetime.utcnow().timestamp()
    T = (expiry_timestamp_ms / 1000 - now) / (60 * 60 * 24 * 365)
    return max(T, 0)

In [32]:
data = [{
        "block_height": 867732,
        "block_hash": "00000000000000000001726dbccd60b22031e392fb9a834fb0e566a24eb06fe8",
        "address": "bc1qxvqhz6q6vn65ppg8a8sf84v6lksdjzg8k8zdqm",
        "tx_id": "61a765dfcf51760c9ba689912a25dc2b7613e60806928b9c2bb235b65321239c",
        "location": "61a765dfcf51760c9ba689912a25dc2b7613e60806928b9c2bb235b65321239c:0:0",
        "output": "61a765dfcf51760c9ba689912a25dc2b7613e60806928b9c2bb235b65321239c:0",
        "value": "294",
        "offset": "0",
        "timestamp": 1730102250000
      },
      {
        "block_height": 867731,
        "block_hash": "0000000000000000000120a60f5b7fc16573ff8048176e87edc22056b44f7cca",
        "address": "bc1q9vyf92nqcjq2r63jg4046lveqrtrdw69sp08dt",
        "tx_id": "2bfd615c9d0e42f5cf5cabf7c5cffe617f963d4eda9b278fc62a9e0ce3f80070",
        "location": "2bfd615c9d0e42f5cf5cabf7c5cffe617f963d4eda9b278fc62a9e0ce3f80070:0:0",
        "output": "2bfd615c9d0e42f5cf5cabf7c5cffe617f963d4eda9b278fc62a9e0ce3f80070:0",
        "value": "546",
        "offset": "0",
        "timestamp": 1730101893000
      },
      {
        "block_height": 867731,
        "block_hash": "0000000000000000000120a60f5b7fc16573ff8048176e87edc22056b44f7cca",
        "address": "bc1q9vyf92nqcjq2r63jg4046lveqrtrdw69sp08dt",
        "tx_id": "2bfd615c9d0e42f5cf5cabf7c5cffe617f963d4eda9b278fc62a9e0ce3f80070",
        "location": "2bfd615c9d0e42f5cf5cabf7c5cffe617f963d4eda9b278fc62a9e0ce3f80070:0:0",
        "output": "2bfd615c9d0e42f5cf5cabf7c5cffe617f963d4eda9b278fc62a9e0ce3f80070:0",
        "value": "800",
        "offset": "0",
        "timestamp": 1730102893000
      }
      ]
sigma = estimate_volatility(data)
print(sigma)

[546. 294. 800.]
[ 0.          5.95       16.66666667]
[-0.61903921  1.00103196]
[ 5.95       10.71666667]
[-0.25378133  0.30578634]
5.3452627007328966


In [ ]:
#Rebar data feeds

REBAR_URL = "https://api.rebarlabs.io"
headers = {
        "X-API-Key":REBAR_API_KEY
    }

#inscriptions
def get_inscriptions(offset:int=0, limit:int=60):
    url =  f"{REBAR_URL}/ordinals/v1/inscriptions?offset={offset}&limit={limit}"
    res = requests.get(url,headers=headers)
    return res.json()

def get_transfers_for_inscription(inscription_id:str):
    url =  f"{REBAR_URL}/ordinals/v1/inscriptions/{inscription_id}/transfers"
    res = requests.get(url,headers=headers)
    return res.json()

def get_prices_for_inscription(inscription_id:str):
    transfers= get_transfers_for_inscription(inscription_id)["results"]
    prices = [transfer["value"] for transfer in transfers]
    print(prices)
    return prices

def get_prices_for_brc_20(ticker, limit=20):
    """
    Fetches recent inferred prices from BRC-20 token activity.

    Args:
        ticker (str): BRC-20 token ticker.
        limit (int): Number of recent events to fetch.

    Returns:
        List[float]: Inferred historical prices (chronological order).
    """
    url = f"{REBAR_URL}/ordinals/v1/brc-20/activity"
    params = {"ticker": ticker, "limit": limit}
    response = requests.get(url, headers=headers, params=params).json()

    prices = []
    for tx in response.get("results", []):
        # You must customize this logic depending on how "price" is inferred
        if "price" in tx:
            prices.append(float(tx["price"]))  # Placeholder key
    return prices[::-1]


def price_brc20_option(ticker, strike_price, expiry_timestamp_ms, option_type='call', risk_free_rate=0.0):
    """
    Calculates the theoretical price of a BRC-20 option.

    Args:
        ticker (str): The BRC-20 token ticker.
        strike_price (float): Strike price of the option.
        expiry_timestamp_ms (int): Expiry timestamp in milliseconds.
        option_type (str): 'call' or 'put'.
        risk_free_rate (float): Risk-free rate (default is 0.0).

    Returns:
        float: Option price.
    """
    prices = get_prices_for_brc_20(ticker)
    if len(prices) < 2:
        raise ValueError("Not enough price data to estimate volatility.")

    spot_price = prices[-1]
    sigma = estimate_volatility(prices)
    T = time_to_expiry(expiry_timestamp_ms)

    return black_scholes_price(S=spot_price, K=strike_price, T=T, r=risk_free_rate, sigma=sigma, option_type=option_type)

def price_inscription_option(id, strike_price, expiry_timestamp_ms, option_type='call', risk_free_rate=0.0):
    """
    Calculates the theoretical price of an inscription option.

    Args:
        id (str): The id ticker.
        strike_price (float): Strike price of the option.
        expiry_timestamp_ms (int): Expiry timestamp in milliseconds.
        option_type (str): 'call' or 'put'.
        risk_free_rate (float): Risk-free rate (default is 0.0).

    Returns:
        float: Option price.
    """
    prices = get_prices_for_inscription(id)
    if len(prices) < 2:
        raise ValueError("Not enough price data to estimate volatility.")

    spot_price = prices[-1]
    sigma = estimate_volatility(prices)
    T = time_to_expiry(expiry_timestamp_ms)

    return black_scholes_price(S=spot_price, K=strike_price, T=T, r=risk_free_rate, sigma=sigma, option_type=option_type)


In [33]:
for i in range(10):
    results = get_inscriptions(i*60)["results"]
    inscriptions_df = pd.DataFrame(results)
    inscriptions_df.head()
    inscriptions_df.shape

    transfer_array = []
    for index, row in inscriptions_df.iterrows():
        transfers = get_transfers_for_inscription(row["id"])["results"]
        transfer_array.append({
            "id": row["id"],  # optional, but often helpful for mapping
            "count": len(transfers),
            "transfers": transfers
        })

    sorted_transfer_array = sorted(transfer_array, key=lambda x: x["count"], reverse=True)
    with open(f"sorted_transfer_data_{i}.json", "w") as f:
        json.dump(sorted_transfer_array, f, indent=2)



In [ ]:
random_ids = inscriptions_df.sample(5, axis=0)['id']
id=random_ids.iloc[0]

transfers= get_transfers_for_inscription(id)["results"]
print(transfers)

prices = get_prices_for_inscription(id)

[{'block_height': 862701, 'block_hash': '00000000000000000000c5d4ae1736dfe247a96b51979000a894fd48e2bb49e8', 'address': 'bc1pqv6f96fpc7ng7kg7kaw9kjm0jmwrk5xfyffzz0scjcuhn85ke86s5v3lwy', 'tx_id': 'e5a8f283fc3903bf236120a75d2290217405a317156d5e9635227c264b47ea7a', 'location': 'e5a8f283fc3903bf236120a75d2290217405a317156d5e9635227c264b47ea7a:0:0', 'output': 'e5a8f283fc3903bf236120a75d2290217405a317156d5e9635227c264b47ea7a:0', 'value': '546', 'offset': '0', 'timestamp': 1727202250000}]
['546']


In [ ]:
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    """
    Calculate the Black-Scholes price of a European call or put option.

    Parameters:
        S (float): Current price of the underlying asset (spot price).
        K (float): Strike price of the option.
        T (float): Time to expiration in years.
        r (float): Annual risk-free interest rate (as a decimal).
        sigma (float): Volatility of the underlying asset (standard deviation of returns).
        option_type (str): Type of the option, either 'call' or 'put'. Defaults to 'call'.

    Returns:
        float: Theoretical price of the option using the Black-Scholes model.

    Notes:
        - If T <= 0 or sigma <= 0, the function returns the intrinsic value of the option.
        - Assumes no dividends and efficient markets.
    """
    if T <= 0 or sigma <= 0:
        return max(0.0, S - K) if option_type == 'call' else max(0.0, K - S)

    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type == 'call':
        return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    else:
        return K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def vega(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return 0.0
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    return S * norm.pdf(d1) * math.sqrt(T)

def implied_volatility(option_price, S, K, T, r, option_type='call'):
    def objective(sigma):
        return black_scholes_price(S, K, T, r, sigma, option_type) - option_price
    try:
        # Safe bounded root-finding
        return brentq(objective, 1e-6, 3.0, maxiter=1000)
    except (ValueError, RuntimeError):
        return float('nan')